In [ ]:
%%time
# --------------------------------------------------------------
# The original code is the following file created by google.
#
# object_detection_tutorial.ipynb
# Copyright 2018 The TensorFlow Authors. All Rights Reserved.
# --------------------------------------------------------------
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
print(tf.__version__)

import numpy as np
from PIL import Image
from PIL import ImageDraw

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile('face.pb', 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
    
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)
    
def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:  
            
            # Output tensor name 
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 
                'detection_scores','detection_classes',
            ]:
                tensor_name = key + ':0'
                tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)

            # Input tensor name
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
          
            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})
            
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
    return output_dict

# Run
image = Image.open('./images/image1.jpg')
image_np = load_image_into_numpy_array(image)
output_dict = run_inference_for_single_image(image_np, detection_graph)

# Rectangle drawing
width,height = image.size
d = ImageDraw.Draw(image)
for i in range(output_dict['num_detections']):
    if(output_dict['detection_scores'][i] > 0.10):
        top,left,bottom,right = (output_dict['detection_boxes'][i])        

        # When saving
        # image.crop((left*width,top*height,right*width,bottom*height)).save('output'+ str(i+1) +'.jpg')
        
        # Drawing
        d.rectangle([(left*width,top*height), (right*width, bottom*height)], outline='green',  width=3)
        
image